# Prediction by Diagnosis Codes
In this notebook we use some fancier networks. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import os, sys

%matplotlib inline

In [2]:
path = '/nfs/turbo/intmed-bnallamo-turbo/wsliu/Data/NRD/'

In [2]:
path = '/nfs/turbo/umms-awaljee/wsliu/Data/NRD/'

In [3]:
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [4]:
from keras.layers import Input, Embedding, concatenate, Reshape, BatchNormalization, add, LSTM, CuDNNLSTM, CuDNNGRU, Lambda
from keras.models import Model
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical
from keras.optimizers import Adam
import keras.backend as K

Using TensorFlow backend.


In [5]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from DL_utils import plot_roc
from keras_addon import AUCCheckPoint

## Data Preparation

In [6]:
core_dtypes_pd = {'AGE': float,
 'AWEEKEND': float,
 'DIED': float,
 'DISCWT': float,
 'DISPUNIFORM': float,
 'DMONTH': float,
 'DQTR': float,
 'DRG': float,
 'DRGVER': float,
 'DRG_NoPOA': float,
 'DX1': bytes,
 'DX10': bytes,
 'DX11': bytes,
 'DX12': bytes,
 'DX13': bytes,
 'DX14': bytes,
 'DX15': bytes,
 'DX16': bytes,
 'DX17': bytes,
 'DX18': bytes,
 'DX19': bytes,
 'DX2': bytes,
 'DX20': bytes,
 'DX21': bytes,
 'DX22': bytes,
 'DX23': bytes,
 'DX24': bytes,
 'DX25': bytes,
 'DX26': bytes,
 'DX27': bytes,
 'DX28': bytes,
 'DX29': bytes,
 'DX3': bytes,
 'DX30': bytes,
 'DX4': bytes,
 'DX5': bytes,
 'DX6': bytes,
 'DX7': bytes,
 'DX8': bytes,
 'DX9': bytes,
 'DXCCS1': float,
 'DXCCS10': float,
 'DXCCS11': float,
 'DXCCS12': float,
 'DXCCS13': float,
 'DXCCS14': float,
 'DXCCS15': float,
 'DXCCS16': float,
 'DXCCS17': float,
 'DXCCS18': float,
 'DXCCS19': float,
 'DXCCS2': float,
 'DXCCS20': float,
 'DXCCS21': float,
 'DXCCS22': float,
 'DXCCS23': float,
 'DXCCS24': float,
 'DXCCS25': float,
 'DXCCS26': float,
 'DXCCS27': float,
 'DXCCS28': float,
 'DXCCS29': float,
 'DXCCS3': float,
 'DXCCS30': float,
 'DXCCS4': float,
 'DXCCS5': float,
 'DXCCS6': float,
 'DXCCS7': float,
 'DXCCS8': float,
 'DXCCS9': float,
 'ECODE1': bytes,
 'ECODE2': bytes,
 'ECODE3': bytes,
 'ECODE4': bytes,
 'ELECTIVE': float,
 'E_CCS1': float,
 'E_CCS2': float,
 'E_CCS3': float,
 'E_CCS4': float,
 'FEMALE': float,
 'HCUP_ED': float,
 'HOSP_NRD': float,
 'KEY_NRD': float,
 'LOS': float,
 'MDC': float,
 'MDC_NoPOA': float,
 'NCHRONIC': float,
 'NDX': float,
 'NECODE': float,
 'NPR': float,
 'NRD_DaysToEvent': float,
 'NRD_STRATUM': float,
 'NRD_VisitLink': bytes,
 'ORPROC': float,
 'PAY1': float,
 'PL_NCHS': float,
 'PR1': bytes,
 'PR10': bytes,
 'PR11': bytes,
 'PR12': bytes,
 'PR13': bytes,
 'PR14': bytes,
 'PR15': bytes,
 'PR2': bytes,
 'PR3': bytes,
 'PR4': bytes,
 'PR5': bytes,
 'PR6': bytes,
 'PR7': bytes,
 'PR8': bytes,
 'PR9': bytes,
 'PRCCS1': float,
 'PRCCS10': float,
 'PRCCS11': float,
 'PRCCS12': float,
 'PRCCS13': float,
 'PRCCS14': float,
 'PRCCS15': float,
 'PRCCS2': float,
 'PRCCS3': float,
 'PRCCS4': float,
 'PRCCS5': float,
 'PRCCS6': float,
 'PRCCS7': float,
 'PRCCS8': float,
 'PRCCS9': float,
 'PRDAY1': float,
 'PRDAY10': float,
 'PRDAY11': float,
 'PRDAY12': float,
 'PRDAY13': float,
 'PRDAY14': float,
 'PRDAY15': float,
 'PRDAY2': float,
 'PRDAY3': float,
 'PRDAY4': float,
 'PRDAY5': float,
 'PRDAY6': float,
 'PRDAY7': float,
 'PRDAY8': float,
 'PRDAY9': float,
 'REHABTRANSFER': float,
 'RESIDENT': float,
 'SAMEDAYEVENT': bytes,
 'SERVICELINE': float,
 'TOTCHG': float,
 'YEAR': float,
 'ZIPINC_QRTL': float}

In [7]:
ami_index = pd.read_csv(path+'cohorts/ami_index.csv', dtype=core_dtypes_pd)

train_comorb = pd.read_csv(path+'cohorts/ami/comorb_train.csv')
test_comorb = pd.read_csv(path+'cohorts/ami/comorb_test.csv')

train_df = ami_index[ami_index['KEY_NRD'].isin(train_comorb['KEY_NRD'])]
tst_df = ami_index[ami_index['KEY_NRD'].isin(test_comorb['KEY_NRD'])]

N_train = len(train_df)
N_tst = len(tst_df)
all_df = pd.concat([train_df, tst_df])

del(ami_index, train_comorb, test_comorb)

In [8]:
trn_df, val_df = train_test_split(train_df, test_size=0.11, stratify=train_df.HOSP_NRD)
N_trn = len(trn_df)
N_val = len(val_df)

train_df = pd.concat([trn_df, val_df])

Define the dictionaries for DX, DX1 and hosp, from value to int. 

In [121]:
N_DX = 29
DXs = ['DX'+str(n) for n in range(2, N_DX+2)]

DX_series = pd.concat([all_df[DX] for DX in DXs])
DX_series = DX_series.fillna('missing')

In [122]:
code_freq = DX_series.value_counts()

rare_code = code_freq[code_freq<3].index

DX_series.loc[DX_series.isin(rare_code)] = 'rare'

In [126]:
DX_series = DX_series.astype('category')

DX_cat = DX_series.cat.categories
n_DX_cat = len(DX_cat)
DX_series = DX_series.cat.rename_categories(range(n_DX_cat))

DX_dict = dict(zip(DX_cat, range(n_DX_cat)))

In [128]:
DX1_series = all_df['DX1'].astype('category')
DX1_cat = DX1_series.cat.categories
DX1_int_cat = range(len(DX1_cat))

DX1_dict = dict(zip(DX1_cat, DX1_int_cat))

In [130]:
hosp_series = all_df['HOSP_NRD'].astype('category')
hosp_cat = hosp_series.cat.categories

hosp_dict = dict(zip(hosp_cat, range(len(hosp_cat))))

## LSTM with Raw ICD9 codes

In [9]:
N_DX = 29
DXs = ['DX'+str(n) for n in range(2, N_DX+2)]

In [134]:
DX_df = train_df[DXs]
DX_df = DX_df.fillna('missing')
DX_df[DX_df.isin(rare_code)] = 'rare'

In [138]:
DX_df = DX_df.replace(DX_dict)

In [146]:
DX_df = DX_df.reset_index(drop=True)

In [142]:
DX_mat_trn = DX_df.values[:N_trn, ]
DX_mat_val = DX_df.values[N_trn:, ]

In [145]:
DX_df.head()

,index,DX2,DX3,DX4,DX5,DX6,DX7,DX8,DX9,DX10,...,DX21,DX22,DX23,DX24,DX25,DX26,DX27,DX28,DX29,DX30
0,17497,519,419,1345,1346,577,1282,1419,626,3749,...,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749
1,43355,1419,1405,3097,3107,559,1783,2853,2137,2701,...,2524,3056,3749,3749,3749,3749,3749,3749,3749,3749
2,173075,3477,406,1385,1393,1404,519,1345,3749,3749,...,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749
3,147543,3561,1868,3717,1339,1766,478,966,419,1344,...,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749
4,161224,3593,2076,627,626,1294,2088,2114,3406,3544,...,3749,3749,3749,3749,3749,3749,3749,3749,3749,3749


In [10]:
DX_series = pd.concat([train_df[DX] for DX in DXs])
DX_series = DX_series.fillna('missing')
DX_series = DX_series.astype('category')

DX_cat = DX_series.cat.categories
n_DX_cat = len(DX_cat)
DX_series = DX_series.cat.rename_categories(range(n_DX_cat))

DX_mat = np.transpose(DX_series.astype(int).values.reshape(N_DX, len(train_df)))

DX_dict = dict(zip(DX_cat, range(n_DX_cat)))

In [11]:
DX_mat_trn = DX_mat[:N_trn, ]
DX_mat_val = DX_mat[N_trn:, ]

In [38]:
DX_mat_val[:5, ]

array([[2578, 2941, 1952, 3282,  804, 5327, 5327, 5327, 5327, 5327, 5327,
        5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327,
        5327, 5327, 5327, 5327, 5327, 5327, 5327],
       [5082, 5160, 1952, 4934, 2862, 5327, 5327, 5327, 5327, 5327, 5327,
        5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327,
        5327, 5327, 5327, 5327, 5327, 5327, 5327],
       [ 926, 2578, 3785, 3789,  679, 2045, 5163, 4645, 4906, 4959, 2048,
        1885,  868, 5286, 1374, 4967, 1952,  842, 4726,  804,   66, 2878,
        5327, 5327, 5327, 5327, 5327, 5327, 5327],
       [2034, 2858, 1896, 2045, 2050, 1952,  804, 1975, 1997, 1950, 1966,
        1953, 5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327, 5327,
        5327, 5327, 5327, 5327, 5327, 5327, 5327],
       [2041, 2084, 4919, 2578, 2080, 1891, 5160, 2402, 4934, 4941, 2485,
        3440, 2203, 5059,  804, 4942,  681, 2045, 1951, 1966, 4959, 2050,
        5060, 5327, 5327, 5327, 5327, 5327, 5327]])

In [13]:
DX_ohe = np.zeros((len(train_df), n_DX_cat))

for j in range(DX_mat.shape[0]):
    for i in range(DX_mat.shape[1]):
        DX_ohe[j, DX_mat[j, i]] = 1

DX_ohe_trn = DX_ohe[:N_trn, ]
DX_ohe_val = DX_ohe[N_trn:, ]

In [15]:
demo_mat = train_df[['AGE', 'FEMALE']].values

age_mean = train_df['AGE'].mean()
age_std = train_df['AGE'].std()

demo_mat[:, 0] = (demo_mat[:, 0]-age_mean)/age_std

demo_mat_trn = demo_mat[:N_trn, ]
demo_mat_val = demo_mat[N_trn:, ]

In [86]:
hosp_series = train_df['HOSP_NRD'].astype('category')
hosp_cat = hosp_series.cat.categories
hosp_series = hosp_series.cat.rename_categories(range(len(hosp_cat)))
hosp_array = hosp_series.astype(int).values

hosp_dict = dict(zip(hosp_cat, hosp_series.cat.categories))

hosp_array_trn = hosp_array[:N_trn]
hosp_array_val = hosp_array[N_trn:]

In [100]:
DX1_series = train_df['DX1'].astype('category')
DX1_cat = DX1_series.cat.categories
DX1_int_cat = range(len(DX1_cat))

DX1_dict = dict(zip(DX1_cat, DX1_int_cat))

DX1_mat = np.zeros((len(DX1_series), len(DX1_cat)))

for i, dx1 in enumerate(DX1_series.values):
    DX1_mat[i, DX1_dict[dx1]] = 1

DX1_mat_trn = DX1_mat[:N_trn, ]
DX1_mat_val = DX1_mat[N_trn:, ]

In [18]:
y = train_df['readm30'].values.astype(int)

Y_trn = to_categorical(y[:N_trn])
Y_val = to_categorical(y[N_trn:])

### Model Building with Embedding

In [46]:
DX_embed_dim = 50
hosp_embed_dim = 1

In [66]:
input_DX = Input(shape = (N_DX,))
DX_embed = Embedding(input_dim=n_DX_cat, output_dim=DX_embed_dim, input_length=N_DX)(input_DX)

#DX_out, _, DX_feature = CuDNNLSTM(DX_embed_dim, return_sequences=False, return_state=True)(DX_embed)
#DX_feature = Lambda(lambda x: K.sum(x, axis=1))(DX_embed)
DX_feature = CuDNNLSTM(DX_embed_dim, return_sequences=False)(DX_embed)

input_demo = Input(shape=(2, ))

input_DX1 = Input(shape=(len(DX1_cat),))

input_hosp = Input(shape=(1,))
hosp_embed = Embedding(input_dim=len(hosp_cat), output_dim=hosp_embed_dim, input_length=1)(input_hosp)
hosp_embed = Reshape((hosp_embed_dim, ))(hosp_embed)

merged = concatenate([input_demo, input_DX1, DX_feature, hosp_embed], axis=1)

x = Dense(128, activation='relu')(merged)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)

prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=[input_demo, input_DX1, input_DX, input_hosp], outputs=prediction)

In [68]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 29)           0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 29, 50)       266400      input_17[0][0]                   
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1, 1)         869         input_20[0][0]                   
__________________________________________________________________________________________________
input_18 (

In [69]:
adam = Adam(lr=0.0002)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [70]:
checkpoint = ModelCheckpoint(filepath=model_path+'ami_icd9_lstm_valloss1.h5', save_best_only=True, save_weights_only=True)
auccheckpoint = AUCCheckPoint(filepath=model_path+'ami_icd9_lstm_auc1.h5', validation_y=Y_val[:, 1], 
                             validation_x=[demo_mat_val, DX1_mat_val, DX_mat_val, hosp_array_val])
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=1.e-8)
earlystop = EarlyStopping(monitor='val_loss', patience=20)

In [71]:
class_weight = {0:(Y_trn.shape[0]/sum(Y_trn[:, 0])), 1:(Y_trn.shape[0]/sum(Y_trn[:, 1]))}

In [72]:
hist = model.fit([demo_mat_trn, DX1_mat_trn, DX_mat_trn, hosp_array_trn], Y_trn, 
                 batch_size=512, epochs=20, callbacks=[checkpoint, auccheckpoint, reduce_lr, earlystop], class_weight=class_weight, 
                 validation_data=[[demo_mat_val, DX1_mat_val, DX_mat_val, hosp_array_val], Y_val], 
                verbose=2)

Train on 145667 samples, validate on 18004 samples
Epoch 1/20
 - 5s - loss: 1.3403 - acc: 0.5801 - val_loss: 0.5770 - val_acc: 0.6839
AUC: 0.6875

Epoch 2/20
 - 2s - loss: 1.2773 - acc: 0.6194 - val_loss: 0.6305 - val_acc: 0.6206
AUC: 0.6927

Epoch 3/20
 - 2s - loss: 1.2626 - acc: 0.6263 - val_loss: 0.5989 - val_acc: 0.6489
AUC: 0.6914

Epoch 4/20
 - 2s - loss: 1.2523 - acc: 0.6295 - val_loss: 0.6561 - val_acc: 0.6030
AUC: 0.6892

Epoch 5/20
 - 2s - loss: 1.2477 - acc: 0.6364 - val_loss: 0.5904 - val_acc: 0.6597
AUC: 0.6876

Epoch 6/20
 - 2s - loss: 1.2396 - acc: 0.6391 - val_loss: 0.6432 - val_acc: 0.6033
AUC: 0.6827

Epoch 7/20
 - 2s - loss: 1.2340 - acc: 0.6403 - val_loss: 0.5949 - val_acc: 0.6571
AUC: 0.6842

Epoch 8/20
 - 2s - loss: 1.2275 - acc: 0.6393 - val_loss: 0.6144 - val_acc: 0.6297
AUC: 0.6789

Epoch 9/20
 - 2s - loss: 1.2207 - acc: 0.6413 - val_loss: 0.6658 - val_acc: 0.5938
AUC: 0.6780

Epoch 10/20
 - 2s - loss: 1.2146 - acc: 0.6426 - val_loss: 0.6117 - val_acc: 0.6355
A

### Model testing

In [76]:
tst_df_temp = tst_df.copy()

In [82]:
DX_df_tst = tst_df_temp[DXs].replace(DX_dict)

In [84]:
DX_mat_tst = DX_df_tst.values

In [105]:
DX_dict['V501']

KeyError: 'V501'

In [104]:
DX_mat_tst

array([[1189, 1952, 1885, ..., nan, nan, nan],
       [5161, 1885, 679, ..., nan, nan, nan],
       [804, 1885, 1952, ..., nan, nan, nan],
       ...,
       [2034, 5160, 4906, ..., nan, nan, nan],
       [4934, 4921, 890, ..., nan, nan, nan],
       [1952, 679, 1049, ..., nan, nan, nan]], dtype=object)

In [85]:
demo_mat_tst = tst_df[['AGE', 'FEMALE']].values
demo_mat_tst[:, 0] = (demo_mat_tst[:, 0]-age_mean)/age_std

In [91]:
hosp_array_tst = tst_df['HOSP_NRD'].replace(hosp_dict).astype(int).values

In [97]:
DX1_mat_tst = np.zeros((len(tst_df), len(DX1_cat)))

for i, dx1 in enumerate(tst_df.DX1.values):
    DX1_mat_tst[i, DX1_dict[dx1]] = 1

In [101]:
model.load_weights(model_path+'ami_icd9_lstm_auc1.h5')

In [102]:
y = model.predict([demo_mat_tst, DX1_mat_tst, DX_mat_tst, hosp_array_tst], verbose=1)

 1792/18186 [=>............................] - ETA: 1s

ValueError: could not convert string to float: 'V501'

### Model Building with OHE

In [20]:
DX_embed_dim = 50
hosp_embed_dim = 1

In [55]:
input_DX = Input(shape = (n_DX_cat,))
DX_feature = Dense(DX_embed_dim, use_bias=False)(input_DX)

input_demo = Input(shape=(2, ))

input_DX1 = Input(shape=(len(DX1_cat),))

input_hosp = Input(shape=(1,))
hosp_embed = Embedding(input_dim=len(hosp_cat), output_dim=hosp_embed_dim, input_length=1)(input_hosp)
hosp_embed = Reshape((hosp_embed_dim, ))(hosp_embed)

merged = concatenate([input_demo, input_DX1, DX_feature, hosp_embed], axis=1)

x = Dense(128, activation='relu')(merged)
#x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
#x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)

prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=[input_demo, input_DX1, input_DX, input_hosp], outputs=prediction)

In [56]:
DX_feature.shape

TensorShape([Dimension(None), Dimension(50)])

In [58]:
adam = Adam(lr=0.0002)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
checkpoint = ModelCheckpoint(filepath=model_path+'ami_icd9_ohe_valloss1.h5', save_best_only=True, save_weights_only=True)
auccheckpoint = AUCCheckPoint(filepath=model_path+'ami_icd9_ohe_auc1.h5', validation_y=Y_val[:, 1], 
                             validation_x=[demo_mat_val, DX1_mat_val, DX_ohe_val, hosp_array_val])
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=1.e-8)
earlystop = EarlyStopping(monitor='val_loss', patience=20)

In [60]:
class_weight = {0:(Y_trn.shape[0]/sum(Y_trn[:, 0])), 1:(Y_trn.shape[0]/sum(Y_trn[:, 1]))}

In [61]:
hist = model.fit([demo_mat_trn, DX1_mat_trn, DX_ohe_trn, hosp_array_trn], Y_trn, 
                 batch_size=512, epochs=20, callbacks=[checkpoint, auccheckpoint, reduce_lr, earlystop], class_weight=class_weight, 
                 validation_data=[[demo_mat_val, DX1_mat_val, DX_ohe_val, hosp_array_val], Y_val], 
                verbose=2)

Train on 145667 samples, validate on 18004 samples
Epoch 1/20
 - 7s - loss: 1.3245 - acc: 0.5939 - val_loss: 0.6438 - val_acc: 0.6399
AUC: 0.6854

Epoch 2/20
 - 9s - loss: 1.2781 - acc: 0.6356 - val_loss: 0.6329 - val_acc: 0.6380
AUC: 0.6898

Epoch 3/20
 - 9s - loss: 1.2583 - acc: 0.6404 - val_loss: 0.6276 - val_acc: 0.6399
AUC: 0.6901

Epoch 4/20
 - 9s - loss: 1.2450 - acc: 0.6460 - val_loss: 0.6301 - val_acc: 0.6279
AUC: 0.6871

Epoch 5/20
 - 9s - loss: 1.2293 - acc: 0.6470 - val_loss: 0.6398 - val_acc: 0.6230
AUC: 0.6841

Epoch 6/20
 - 9s - loss: 1.2162 - acc: 0.6539 - val_loss: 0.6214 - val_acc: 0.6377
AUC: 0.6814

Epoch 7/20
 - 9s - loss: 1.2016 - acc: 0.6574 - val_loss: 0.6104 - val_acc: 0.6439
AUC: 0.6778

Epoch 8/20
 - 9s - loss: 1.1875 - acc: 0.6590 - val_loss: 0.6120 - val_acc: 0.6417
AUC: 0.6747

Epoch 9/20
 - 9s - loss: 1.1727 - acc: 0.6650 - val_loss: 0.6097 - val_acc: 0.6387
AUC: 0.6750

Epoch 10/20
 - 9s - loss: 1.1610 - acc: 0.6660 - val_loss: 0.6189 - val_acc: 0.6329
A